<a href="https://colab.research.google.com/github/AthulyaSK/Budget-Optimization/blob/main/06_CapBudget_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

     |████████████████████████████████| 9.7 MB 5.3 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123991 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_4.65-1_amd64.deb ...
Unpacking glpk-utils (4.65-1) ...
Setting up 

#Capital Budgeting Problem (Base)

In [ ]:
#Capital Budgeting Base Problem Data
cost = [5.0,2.4,3.5,5.9,6.9,4.5,3.0]
npv = [5.6,2.7,3.9,6.8,7.7,5.1,3.3]
budget = 15
n = len(cost)

#define the concrete model
model = ConcreteModel()

#DVs
model.x = Var(range(n), domain = Binary) #NOTE: This is where we restrict the decision variables to be binary (could also put things like NonNegativeIntegers)

#budget constraint
model.BudgetConstraint = Constraint(expr = sum(model.x[i]*cost[i] for i in range(n)) <= budget)

#objective
model.Objective = Objective(expr = sum(model.x[i]*npv[i] for i in range(n)), sense = maximize)

In [ ]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpi3hu30ph.glpk.raw --wglp /tmp/tmpvrff00zt.glpk.glp
 --cpxlp /tmp/tmprgsl4wiu.pyomo.lp
Reading problem data from '/tmp/tmprgsl4wiu.pyomo.lp'...
/tmp/tmprgsl4wiu.pyomo.lp:37: warning: lower bound of variable 'x1' redefined
/tmp/tmprgsl4wiu.pyomo.lp:37: warning: upper bound of variable 'x1' redefined
2 rows, 8 columns, 8 non-zeros
7 integer variables, all of which are binary
44 lines were read
Writing problem data to '/tmp/tmpvrff00zt.glpk.glp'...
31 lines were written
GLPK Integer Optimizer, v4.65
2 rows, 8 columns, 8 non-zeros
7 integer variables, all of which are binary
Preprocessing...
1 row, 7 columns, 7 non-zeros
7 integer variables, all of which are binary
Scaling...
 A: min|aij| =  2.400e+00  max|aij| =  6.900e+00  ratio =  2.875e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
Solving LP relaxation...
GLPK Simplex

In [ ]:
#print out solution
print("invest in options: ")
for i in range(n):
    if model.x[i]() == 1:
        print(i)

print("Total NPV:", model.Objective())
print("Total Cost:", model.BudgetConstraint())

invest in options: 
1
2
3
6
Total NPV: 16.7
Total Cost: 14.8


#A.	You must invest in investment 0 or 4 (or both).

In [ ]:
#Capital Budgeting Base Problem Data
cost = [5.0,2.4,3.5,5.9,6.9,4.5,3.0]
npv = [5.6,2.7,3.9,6.8,7.7,5.1,3.3]
budget = 15
n = len(cost)

#define the concrete model
model = ConcreteModel()

model.x = Var(range(n), domain = Binary) #NOTE: This is where we restrict the decision variables to be binary (could also put things like NonNegativeIntegers)

model.BudgetConstraint = Constraint(expr = sum(model.x[i]*cost[i] for i in range(n)) <= budget)
model.BudgetConstraint2 = Constraint(expr = model.x[0] + model.x[4] >= 1)

#objective
model.Objective1 = Objective(expr = sum(model.x[i]*npv[i] for i in range(n)), sense = maximize)

model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {0, 1, 2, 3, 4, 5, 6}

1 Var Declarations
    x : Size=7, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    Objective1 : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5.6*x[0] + 2.7*x[1] + 3.9*x[2] + 6.8*x[3] + 7.7*x[4] + 5.1*x[5] + 3.3*x[6]

2 Constraint Declarations
    BudgetConstraint : Si

In [ ]:
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpstxatiey.glpk.raw --wglp /tmp/tmptfl1wf79.glpk.glp
 --cpxlp /tmp/tmp9flyq2ki.pyomo.lp
Reading problem data from '/tmp/tmp9flyq2ki.pyomo.lp'...
/tmp/tmp9flyq2ki.pyomo.lp:42: warning: lower bound of variable 'x1' redefined
/tmp/tmp9flyq2ki.pyomo.lp:42: warning: upper bound of variable 'x1' redefined
3 rows, 8 columns, 10 non-zeros
7 integer variables, all of which are binary
49 lines were read
Writing problem data to '/tmp/tmptfl1wf79.glpk.glp'...
35 lines were written
GLPK Integer Optimizer, v4.65
3 rows, 8 columns, 10 non-zeros
7 integer variables, all of which are binary
Preprocessing...
1 hidden packing inequaliti(es) were detected
2 rows, 7 columns, 9 non-zeros
7 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  6.900e+00  ratio =  6.900e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangula

In [ ]:
#print out solution
print("invest in options: ")
for i in range(n):
    if model.x[i]() == 1:
        print(i)

print("Total NPV:", model.Objective())
print("Total Cost:", model.BudgetConstraint())

invest in options: 
2
4
5
Total NPV: <pyomo.core.base.block._generic_component_decorator object at 0x7fd8af7e0a50>
Total Cost: 14.9


#B.	If you invest in 0, you also have to invest in 5. You can invest in 5 without investing in 0. 

In [ ]:
#Capital Budgeting Base Problem Data
cost = [5.0,2.4,3.5,5.9,6.9,4.5,3.0]
npv = [5.6,2.7,3.9,6.8,7.7,5.1,3.3]
budget = 15
n = len(cost)

#define the concrete model
model = ConcreteModel()

model.x = Var(range(n), domain = Binary) #NOTE: This is where we restrict the decision variables to be binary (could also put things like NonNegativeIntegers)

model.BudgetConstraint = Constraint(expr = sum(model.x[i]*cost[i] for i in range(n)) <= budget)
model.BudgetConstraint2 = Constraint(expr = model.x[0] - model.x[5] <= 0)

#objective
model.Objective1 = Objective(expr = sum(model.x[i]*npv[i] for i in range(n)), sense = maximize)

model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {0, 1, 2, 3, 4, 5, 6}

1 Var Declarations
    x : Size=7, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    Objective1 : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 5.6*x[0] + 2.7*x[1] + 3.9*x[2] + 6.8*x[3] + 7.7*x[4] + 5.1*x[5] + 3.3*x[6]

2 Constraint Declarations
    BudgetConstraint : Si

In [ ]:
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .01 #specifies the optimality gap tolerance (.01 means alg can stop if guarenteed within <1% of optimal obj)
results = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.01 --write /tmp/tmpu4a5wf6z.glpk.raw --wglp /tmp/tmp55b4y6z9.glpk.glp
 --cpxlp /tmp/tmp510arpxl.pyomo.lp
Reading problem data from '/tmp/tmp510arpxl.pyomo.lp'...
/tmp/tmp510arpxl.pyomo.lp:42: warning: lower bound of variable 'x1' redefined
/tmp/tmp510arpxl.pyomo.lp:42: warning: upper bound of variable 'x1' redefined
3 rows, 8 columns, 10 non-zeros
7 integer variables, all of which are binary
49 lines were read
Writing problem data to '/tmp/tmp55b4y6z9.glpk.glp'...
35 lines were written
GLPK Integer Optimizer, v4.65
3 rows, 8 columns, 10 non-zeros
7 integer variables, all of which are binary
Preprocessing...
2 rows, 7 columns, 9 non-zeros
7 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  6.900e+00  ratio =  6.900e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
Solving LP relaxation...
GLPK Simp

In [ ]:
#print out solution
print("invest in options: ")
for i in range(n):
    if model.x[i]() == 1:
        print(i)

print("Total NPV:", model.Objective())
print("Total Cost:", model.BudgetConstraint())

invest in options: 
1
2
3
6
Total NPV: <pyomo.core.base.block._generic_component_decorator object at 0x7f02dd738490>
Total Cost: 14.8
